## Rede **U-net** 
Criada com base no texto [U-net Unleashed: A step-by-step guide on implementing and training your own segmentation model in Tensorflow](https://medium.com/@vipul.sarode007/u-net-unleashed-a-step-by-step-guide-on-implementing-and-training-your-own-segmentation-model-in-a38741776968). Começo 12/01/2026

In [1]:
# Bibliotecas
import os
import cv2
import random
import pandas as pd
# from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt



import tensorflow as tf
from keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Conv2DTranspose, concatenate, Input, Flatten, Dropout
from tensorflow.keras.models import Model

In [3]:
# Encoder
def encoder_block(filters, inputs):
    x = Conv2D(filters, kernel_size = (3,3), padding = 'same', strides = 1, activation = 'relu')(inputs)
    s = Conv2D(filters, kernel_size = (3,3), padding = 'same', strides = 1, activation = 'relu')(x)
    p = MaxPooling2D(pool_size = (2, 2), padding = 'same')(s)
    return s, p    

In [4]:
# Baseline
def baseline_layer(filters, inputs):
    x = Conv2D(filters, kernel_size = (3,3), padding = 'same', strides = 1, activation = 'relu')(inputs)
    x = Conv2D(filters, kernel_size = (3,3), padding = 'same', strides = 1, activation = 'relu')(x)
    return x

In [5]:
# Decoder
def decoder_block(filters, connections, inputs):
    x = Conv2DTranspose(filters, kernel_size = (2,2), padding = 'same', activation = 'relu', strides = 2)(inputs)
    skip_connections = concatenate([x, connections], axis = -1)
    x = Conv2D(filters, kernel_size = (2,2), padding = 'same', activation = 'relu')(skip_connections)
    x = Conv2D(filters, kernel_size = (3,3), padding = 'same', activation = 'relu')(x)
    return x

### Rede U-Net

In [ ]:
def unet():
    # Defining the input layer
    inputs = Input(shape = (256, 256, 3))

    # Defining the encoder 
    s1, p1 = encoder_block(64, inputs = inputs)
    s2, p2 = encoder_block(128, inputs = p1)
    s3, p3 = encoder_block(256, inputs = p2)
    s4, p4 = encoder_block(512, inputs = p3)

    # Setting the baseline
    baseline = baseline_layer(1024, p4)

    # Defining the decoder
    d1 = decoder_block(512, connections = s4,  input = baseline)
    d2 = decoder_block(256, connections = s3,  input = d1)
    d3 = decoder_block(128, connections = s2,  input = d2)
    d4 = decoder_block(64,  connections = s1,  input = d3)

    # Output layer
    outputs = Conv2D(1, kernel_size = 1, padding = 'same', activation = 'sigmoid')(d4)

    # Finalizing the model
    model = Model(inputs = inputs, outputs = outputs, name = 'U-Net')

    return model